### API Analysis

![alt text](../images/image.png "Title")
This configuration shows the hourly day-ahead (price of energy until the same time tomorrow) for the last two weeks.
When checking the network traffic for the above dates and for the hourly resolution, you will find three .json files being fetched from the API.

A request to the api has the following structure:
https://www.smard.de/app/chart_data/4169/DE/4169_DE_hour_[timestamp_in_milliseconds].json

The following request fetch data for the corresponding time frames.

https://www.smard.de/app/chart_data/4169/DE/4169_DE_hour_1729461600000.json:
Sunday, 6 October 2024 22:00:00 -> Sunday, 13 October 2024 21:00:00

https://www.smard.de/app/chart_data/4169/DE/4169_DE_hour_1728856800000.json:
Sunday, 13 October 2024 22:00:00 -> Sunday, 20 October 2024 21:00:00

https://www.smard.de/app/chart_data/4169/DE/4169_DE_hour_1729461600000.json
Sunday, 20 October 2024 22:00:00 -> Sunday, 27 October 2024 22:00:00


You will find that for example the timestamp 1729461600000 maps to the initial date Sunday, 6 October 2024 22:00:00 and every file contains the date for one week. Interestingly enough the site only shows the data for two weeks even though it had to fetch the data for three entire weeks. If the above links are broken, it may be due to a shift in daylight savings time (DST) which we will have to take into account.

Additionally you will see that each .json file contains around 172 (more or less) time series entries for an entire week.



### Implementing the scraper
We now want to implement a scraper that fetches the hourly energy prices for n amount of days. With the above information we now know that we'll have to find the corresponding timestamps for each week and to fetch the data.

In [6]:
import requests
import numpy as np
import logging
from datetime import datetime, timedelta, timezone
import pytz
import time
from pprint import pprint

In [2]:
logging.basicConfig(level=logging.INFO) 

logger = logging.getLogger("scraper_logger")

# console_handler = logging.StreamHandler()
file_handler = logging.FileHandler("app.log")

# console_handler.setLevel(logging.WARNING)
file_handler.setLevel(logging.WARNING) 

# logger.addHandler(console_handler)
logger.addHandler(file_handler)

In [3]:
def scrape(url, delay):
    response =  requests.get(url)
    response.raise_for_status()

    time.sleep(delay)
    return response

In [4]:
from datetime import datetime, timedelta
import pytz

# Define Berlin timezone
tz_berlin = pytz.timezone("Europe/Berlin")

# Calculate last Monday in Berlin time, taking into account local DST
now = datetime.now(tz_berlin)
days_since_monday = now.weekday()
last_monday_berlin = now - timedelta(days=days_since_monday)
last_monday_berlin = last_monday_berlin.replace(hour=0, minute=0, second=0, microsecond=0)

# Convert Berlin time to UTC and get the timestamp in milliseconds
last_monday_utc = last_monday_berlin.astimezone(pytz.UTC)
last_monday_utc_ms = int(last_monday_utc.timestamp() * 1000)

print("Berlin time (local):", last_monday_berlin)
print("UTC time:", last_monday_utc)
print("UTC timestamp (ms):", last_monday_utc_ms)


Berlin time (local): 2024-12-02 00:00:00+01:00
UTC time: 2024-12-01 23:00:00+00:00
UTC timestamp (ms): 1733094000000


In [5]:
import requests
import logging
from datetime import datetime, timedelta
import numpy as np
import pytz

# Define Berlin timezone
tz_berlin = pytz.timezone("Europe/Berlin")

# Calculate last Monday in Berlin time, taking into account local DST
now = datetime.now(tz_berlin)
days_since_monday = now.weekday()
last_monday_berlin = now - timedelta(days=days_since_monday)
last_monday_berlin = last_monday_berlin.replace(hour=0, minute=0, second=0, microsecond=0)

# Convert Berlin time to UTC and get the timestamp in milliseconds
last_monday_utc = last_monday_berlin.astimezone(pytz.UTC)
last_monday_utc_ms = int(last_monday_utc.timestamp() * 1000)

print("Berlin time (local):", last_monday_berlin)
print("UTC time:", last_monday_utc)
print("UTC timestamp (ms):", last_monday_utc_ms)

# Define constants
week_in_ms = 24 * 60 * 60 * 1000 * 7
delay = 0.5  # seconds
n = 500  # number of weeks
base_url = "https://www.smard.de/app/chart_data/4169/DE/4169_DE_hour_{}.json"

# Use a dictionary to store unique timestamps and prices
energy_ts_data = {}

for k in range(n):
    last_monday_berlin = last_monday_utc.astimezone(tz_berlin)
    last_monday_utc = last_monday_berlin.astimezone(pytz.UTC)
    last_monday_utc_ms = int(last_monday_utc.timestamp() * 1000)

    # Adjust timestamp for daylight savings time (berlin tz) if necessary
    if last_monday_berlin.dst() != timedelta(0):  # DST is in effect
        last_monday_utc_ms -= 60 * 60 * 1000

    try:
        response = requests.get(base_url.format(last_monday_utc_ms))
        response.raise_for_status()
        logging.info(f"Successfully scraped data for ts: {last_monday_berlin} (Europe/Berlin)")
        json_data = response.json()
    except requests.exceptions.HTTPError as http_err:
        logging.warning(f"Failed to scrape data for timestamp: {last_monday_utc} (UTC)\n\tError: {http_err}")
        continue
    except requests.exceptions.JSONDecodeError as decoder_error:
        logging.warning(f"Failed to deserialize JSON: \n\tError: {decoder_error}")
        continue

    # Parse the JSON response
    parsed_json = dict(json_data)

    for ts, price in parsed_json.get("series", []):
        try:
            price_float = float(price)
            # Convert to naive timestamp
            ts_datetime = datetime.fromtimestamp(ts / 1000).replace(tzinfo=None).isoformat()
            print(ts_datetime)
            # Add to the dictionary, overwriting any duplicates
            energy_ts_data[ts_datetime] = price_float
        except TypeError as e:
            logging.warning(f"Failed to parse non-float value for timestamp {ts}\n\tError: {e}")
            continue

    # Move to the previous week
    last_monday_utc = last_monday_utc - timedelta(weeks=1)

# Convert the dictionary to a sorted list of tuples
energy_ts_data_sorted = sorted(energy_ts_data.items())

# Convert to a NumPy array
data = np.array(energy_ts_data_sorted)

print("Final dataset shape:", data.shape)

# Save the data as a CSV file (naive timestamps only)
np.savetxt("../data/day_ahead_energy_prices.csv", data, delimiter=",", fmt="%s")


Berlin time (local): 2024-12-02 00:00:00+01:00
UTC time: 2024-12-01 23:00:00+00:00
UTC timestamp (ms): 1733094000000


INFO:root:Successfully scraped data for ts: 2024-12-02 00:00:00+01:00 (Europe/Berlin)
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'


2024-12-02T00:00:00
2024-12-02T01:00:00
2024-12-02T02:00:00
2024-12-02T03:00:00
2024-12-02T04:00:00
2024-12-02T05:00:00
2024-12-02T06:00:00
2024-12-02T07:00:00
2024-12-02T08:00:00
2024-12-02T09:00:00
2024-12-02T10:00:00
2024-12-02T11:00:00
2024-12-02T12:00:00
2024-12-02T13:00:00
2024-12-02T14:00:00
2024-12-02T15:00:00
2024-12-02T16:00:00
2024-12-02T17:00:00
2024-12-02T18:00:00
2024-12-02T19:00:00
2024-12-02T20:00:00
2024-12-02T21:00:00
2024-12-02T22:00:00
2024-12-02T23:00:00
2024-12-03T00:00:00
2024-12-03T01:00:00
2024-12-03T02:00:00
2024-12-03T03:00:00
2024-12-03T04:00:00
2024-12-03T05:00:00
2024-12-03T06:00:00
2024-12-03T07:00:00
2024-12-03T08:00:00
2024-12-03T09:00:00
2024-12-03T10:00:00
2024-12-03T11:00:00
2024-12-03T12:00:00
2024-12-03T13:00:00
2024-12-03T14:00:00
2024-12-03T15:00:00
2024-12-03T16:00:00
2024-12-03T17:00:00
2024-12-03T18:00:00
2024-12-03T19:00:00
2024-12-03T20:00:00
2024-12-03T21:00:00
2024-12-03T22:00:00
2024-12-03T23:00:00


	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument must be a string or a real number, not 'NoneType'
	Error: float() argument 

2024-11-25T00:00:00
2024-11-25T01:00:00
2024-11-25T02:00:00
2024-11-25T03:00:00
2024-11-25T04:00:00
2024-11-25T05:00:00
2024-11-25T06:00:00
2024-11-25T07:00:00
2024-11-25T08:00:00
2024-11-25T09:00:00
2024-11-25T10:00:00
2024-11-25T11:00:00
2024-11-25T12:00:00
2024-11-25T13:00:00
2024-11-25T14:00:00
2024-11-25T15:00:00
2024-11-25T16:00:00
2024-11-25T17:00:00
2024-11-25T18:00:00
2024-11-25T19:00:00
2024-11-25T20:00:00
2024-11-25T21:00:00
2024-11-25T22:00:00
2024-11-25T23:00:00
2024-11-26T00:00:00
2024-11-26T01:00:00
2024-11-26T02:00:00
2024-11-26T03:00:00
2024-11-26T04:00:00
2024-11-26T05:00:00
2024-11-26T06:00:00
2024-11-26T07:00:00
2024-11-26T08:00:00
2024-11-26T09:00:00
2024-11-26T10:00:00
2024-11-26T11:00:00
2024-11-26T12:00:00
2024-11-26T13:00:00
2024-11-26T14:00:00
2024-11-26T15:00:00
2024-11-26T16:00:00
2024-11-26T17:00:00
2024-11-26T18:00:00
2024-11-26T19:00:00
2024-11-26T20:00:00
2024-11-26T21:00:00
2024-11-26T22:00:00
2024-11-26T23:00:00
2024-11-27T00:00:00
2024-11-27T01:00:00


...

INFO:root:Successfully scraped data for ts: 2024-10-21 01:00:00+02:00 (Europe/Berlin)


2024-10-21T00:00:00
2024-10-21T01:00:00
2024-10-21T02:00:00
2024-10-21T03:00:00
2024-10-21T04:00:00
2024-10-21T05:00:00
2024-10-21T06:00:00
2024-10-21T07:00:00
2024-10-21T08:00:00
2024-10-21T09:00:00
2024-10-21T10:00:00
2024-10-21T11:00:00
2024-10-21T12:00:00
2024-10-21T13:00:00
2024-10-21T14:00:00
2024-10-21T15:00:00
2024-10-21T16:00:00
2024-10-21T17:00:00
2024-10-21T18:00:00
2024-10-21T19:00:00
2024-10-21T20:00:00
2024-10-21T21:00:00
2024-10-21T22:00:00
2024-10-21T23:00:00
2024-10-22T00:00:00
2024-10-22T01:00:00
2024-10-22T02:00:00
2024-10-22T03:00:00
2024-10-22T04:00:00
2024-10-22T05:00:00
2024-10-22T06:00:00
2024-10-22T07:00:00
2024-10-22T08:00:00
2024-10-22T09:00:00
2024-10-22T10:00:00
2024-10-22T11:00:00
2024-10-22T12:00:00
2024-10-22T13:00:00
2024-10-22T14:00:00
2024-10-22T15:00:00
2024-10-22T16:00:00
2024-10-22T17:00:00
2024-10-22T18:00:00
2024-10-22T19:00:00
2024-10-22T20:00:00
2024-10-22T21:00:00
2024-10-22T22:00:00
2024-10-22T23:00:00
2024-10-23T00:00:00
2024-10-23T01:00:00


INFO:root:Successfully scraped data for ts: 2024-10-14 01:00:00+02:00 (Europe/Berlin)


2024-10-14T00:00:00
2024-10-14T01:00:00
2024-10-14T02:00:00
2024-10-14T03:00:00
2024-10-14T04:00:00
2024-10-14T05:00:00
2024-10-14T06:00:00
2024-10-14T07:00:00
2024-10-14T08:00:00
2024-10-14T09:00:00
2024-10-14T10:00:00
2024-10-14T11:00:00
2024-10-14T12:00:00
2024-10-14T13:00:00
2024-10-14T14:00:00
2024-10-14T15:00:00
2024-10-14T16:00:00
2024-10-14T17:00:00
2024-10-14T18:00:00
2024-10-14T19:00:00
2024-10-14T20:00:00
2024-10-14T21:00:00
2024-10-14T22:00:00
2024-10-14T23:00:00
2024-10-15T00:00:00
2024-10-15T01:00:00
2024-10-15T02:00:00
2024-10-15T03:00:00
2024-10-15T04:00:00
2024-10-15T05:00:00
2024-10-15T06:00:00
2024-10-15T07:00:00
2024-10-15T08:00:00
2024-10-15T09:00:00
2024-10-15T10:00:00
2024-10-15T11:00:00
2024-10-15T12:00:00
2024-10-15T13:00:00
2024-10-15T14:00:00
2024-10-15T15:00:00
2024-10-15T16:00:00
2024-10-15T17:00:00
2024-10-15T18:00:00
2024-10-15T19:00:00
2024-10-15T20:00:00
2024-10-15T21:00:00
2024-10-15T22:00:00
2024-10-15T23:00:00
2024-10-16T00:00:00
2024-10-16T01:00:00


KeyboardInterrupt: 

- Weather:
-- wind
-- sun 
-- temp

- per day energy mix
- gas price per day
- 

In [ ]:
start_date = datetime.now()
end_date = datetime(2018, 9, 30)
delta = timedelta(days=1)
delay = 0.2

# end_date = start_date - (10 * delta)

base_url = "https://www.energy-charts.info/charts/energy_pie/data/de/day_pie_{}.json"

current_date = start_date
res = []
while current_date >= end_date:
    try:
        cd_format = current_date.strftime("%Y_%m_%d")
        response = scrape(base_url.format(cd_format), delay)

        logging.info(f"Successfully scraped data for date: {cd_format}")
        res.append((cd_format, response.json()))
    except requests.exceptions.HTTPError as http_err:
        logging.warning(f"Failed to scrape data for date: {cd_format} (UTC)\n\tError: {http_err}")
    except requests.exceptions.JSONDecodeError as decoder_error:
        logging.warning(f"Failed to deserialize JSON: \n\tError: {decoder_error}")
    current_date -= delta


print(len(res))


INFO:root:Successfully scraped data for date: 2024_10_31
INFO:root:Successfully scraped data for date: 2024_10_30
INFO:root:Successfully scraped data for date: 2024_10_29
INFO:root:Successfully scraped data for date: 2024_10_28
INFO:root:Successfully scraped data for date: 2024_10_27
INFO:root:Successfully scraped data for date: 2024_10_26
INFO:root:Successfully scraped data for date: 2024_10_25
INFO:root:Successfully scraped data for date: 2024_10_24
INFO:root:Successfully scraped data for date: 2024_10_23
INFO:root:Successfully scraped data for date: 2024_10_22
INFO:root:Successfully scraped data for date: 2024_10_21
INFO:root:Successfully scraped data for date: 2024_10_20
INFO:root:Successfully scraped data for date: 2024_10_19
INFO:root:Successfully scraped data for date: 2024_10_18
INFO:root:Successfully scraped data for date: 2024_10_17
INFO:root:Successfully scraped data for date: 2024_10_16
INFO:root:Successfully scraped data for date: 2024_10_15
INFO:root:Successfully scraped 

2224


### Energy Mix Scraper

In [ ]:
exclude_cross_boarder_e_trading = True
cbet = "Cross border electricity trading"

dtype = [('date', 'U50'), ('e_component', 'U50'), ('value', 'float32')]

# Initialize an empty structured array
array = np.empty(0, dtype=dtype)

for date, data in res:
    sources = []
    for e_source in data:
        name = str(e_source["name"]["en"])

        if exclude_cross_boarder_e_trading and name == cbet:
            continue

        # Ensure numeric conversion or default to 0
        try:
            y_value = float(e_source["y"])
        except (ValueError, TypeError):
            continue
        
        sources.append((date, name, y_value))
    
    # Convert to a structured array with the correct dtype
    arr = np.array(sources, dtype=dtype)
    
    # Normalize the 'value' column
    arr['value'] /= np.sum(arr['value'], axis=0)

    # Append to the main array
    array = np.append(array, arr)

np.savetxt("../data/daily_market_mix.csv", array, delimiter=",", fmt="%s")
array

array([('2024_10_31', 'Waste renewable', 0.01161428),
       ('2024_10_31', 'Hydro Run-of-River', 0.04549553),
       ('2024_10_31', 'Hydro water reservoir', 0.00522868), ...,
       ('2018_09_30', 'Fossil gas', 0.04589883),
       ('2018_09_30', 'Others', 0.00584625),
       ('2018_09_30', 'Waste non-renewable', 0.0163848 )],
      dtype=[('date', '<U50'), ('e_component', '<U50'), ('value', '<f4')])

In [ ]:
POST /api/raw-data HTTP/1.1
Content-Type: application/json
Accept: */*
Sec-Fetch-Site: cross-site
Accept-Language: en-GB,en;q=0.9
Accept-Encoding: gzip, deflate, br
Sec-Fetch-Mode: cors
Origin: https://www.agora-energiewende.de
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.0.1 Safari/605.1.15
Content-Length: 538
Referer: https://www.agora-energiewende.de/
Connection: keep-alive
Sec-Fetch-Dest: empty
X-Requested-With: XMLHttpRequest
Api-key: agora_live_62ce76dd202927.67115829
Priority: u=3, i


{"filters":{"from":"2023-11-01","to":"2024-10-01","generation":["Total electricity demand","Biomass","Hydro","Wind offshore","Wind onshore","Solar","Total conventional power plant","Nuclear","Lignite","Hard Coal","Natural Gas","Pumped storage generation","Other","Grid emission factor","Total grid emissions","Total Renewables","Total Conventional","Renewable share","Conventional share"]},"x_coordinate":"date_id","y_coordinate":"value","view_name":"live_gen_plus_emi_de_hourly","kpi_name":"power_generation","z_coordinate":"generation"}

SyntaxError: invalid syntax (2008687956.py, line 1)

In [7]:
import requests

# Define the API endpoint and headers
url = "https://api.agora-energy.org/api/raw-data"
headers = {
    "Content-Type": "application/json",
    "Accept": "*/*",
    "Sec-Fetch-Site": "cross-site",
    "Accept-Language": "en-GB,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Sec-Fetch-Mode": "cors",
    "Origin": "https://www.agora-energiewende.de",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.0.1 Safari/605.1.15",
    "Referer": "https://www.agora-energiewende.de/",
    "Connection": "keep-alive",
    "Sec-Fetch-Dest": "empty",
    "X-Requested-With": "XMLHttpRequest",
    "Api-key": "agora_live_62ce76dd202927.67115829",
}

out = []

# Define the payload
for year in [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]:
    payload = {
        "filters": {
            "from": f"{year}-10-01",
            "to": f"{year + 1}-09-30",
            "generation": [
                "Total electricity demand", "Biomass", "Hydro", "Wind offshore",
                "Wind onshore", "Solar", "Total conventional power plant", "Nuclear",
                "Lignite", "Hard Coal", "Natural Gas", "Pumped storage generation",
                "Other", "Grid emission factor", "Total grid emissions", "Total Renewables",
                "Total Conventional", "Renewable share", "Conventional share"
            ]
        },
        "x_coordinate": "date_id",
        "y_coordinate": "value",
        "view_name": "live_gen_plus_emi_de_hourly",
        "kpi_name": "power_generation",
        "z_coordinate": "generation"
    }

    # Make the POST request
    response = requests.post(url, headers=headers, json=payload)

    # Check the response
    if response.status_code == 200:
        print("Request was successful!", year, year+1)
        data = {}
        data = response.json()
        out.extend(data["data"]["data"])
    else:
        print(f"Request failed with status code {response.status_code}", year, year+1)
    time.sleep(0.3)


Request was successful! 2017 2018
Request was successful! 2018 2019
Request was successful! 2019 2020
Request was successful! 2020 2021
Request was successful! 2021 2022
Request was successful! 2022 2023
Request was successful! 2023 2024
Request was successful! 2024 2025


In [8]:
np.savetxt("../data/hourly_market_mix.csv", np.array(out), delimiter=",", fmt="%s")

In [ ]:
data = np.array(out)

mix_categories = [
    "Biomass",
    "Hard Coal",
    "Hydro",
    "Lignite",
    "Natural Gas",
    "Nuclear",
    "Other",
    "Pumped storage generation",
    "Solar",
    "Wind offshore",
    "Wind onshore",
]

other_metrics = [
    "Grid emission factor",
    "Total conventional power plant",
    "Total electricity demand",
    "Total grid emissions",
]

# Define start and end dates as naive datetime objects
start_date = datetime.fromisoformat("2018-10-01T00:00:00")
end_date = datetime.fromisoformat("2024-10-30T00:00:00")

# Generate hourly timestamps
timestamps = [
    start_date + timedelta(hours=i)
    for i in range(int((end_date - start_date).total_seconds() // 3600) + 1)
]
timestamp_strings = [ts.isoformat() for ts in timestamps]
mix_rows = []
other_metrics_rows = []

data_dict = {ts: [] for ts in timestamp_strings}
for d in data:
    d_timestamp = datetime.fromisoformat(d[0]).isoformat()  # Naive datetime conversion
    if d_timestamp in data_dict:
        data_dict[d_timestamp].append(d)
for ts in timestamp_strings:
    hour_data = np.array(data_dict.get(ts, []))  # Fetch data for this timestamp
    if hour_data.size == 0:
        continue

    mix_per_hour = hour_data[np.isin(hour_data[:, 2], mix_categories)]
    
    if mix_per_hour.size == 0:
        continue
    
    mix_per_hour = np.where(mix_per_hour == None, 0.0, mix_per_hour)


    mix_per_hour[:, 1] = (
        mix_per_hour[:, 1].astype(float) / mix_per_hour[:, 1].astype(float).sum()
    )

    row = np.concatenate(([ts], mix_per_hour[:, 1]))
    if row.shape[0] == 11:
        row = np.insert(row, 6, 0.0)
    mix_rows.append(row)

    other_metrics_per_hour = hour_data[np.isin(hour_data[:, 2], other_metrics)]
    if other_metrics_per_hour.size == 0:
        continue
    row = np.concatenate(([ts], other_metrics_per_hour[:, 1]))
    
    other_metrics_rows.append(row)
    
percentage_mix = np.vstack([["Timestamp"] + mix_categories] + mix_rows)
percentage_sources = np.vstack([["Timestamp"] + other_metrics] + other_metrics_rows)

np.savetxt("../data/hourly_market_mix_cleaned.csv", percentage_mix, delimiter=",", fmt="%s")
np.savetxt("../data/hourly_market_metrics_cleaned.csv", percentage_sources, delimiter=",", fmt="%s")

[['Timestamp' 'Biomass' 'Hard Coal' ... 'Solar' 'Wind offshore'
  'Wind onshore']
 ['2018-10-01T00:00:00' 0.09532941429493154 0.14864864864864866 ... 0.0
  0.02957201233063302 0.07835687146031974]
 ['2018-10-01T01:00:00' 0.09473778574630211 0.14298520842671447 ... 0.0
  0.03792021515015689 0.07315105333930973]
 ...
 ['2024-10-29T22:00:00' 0.09587167371300673 0.15058443173339967 ... 0.0
  0.035148802122191826 0.09744673795904833]
 ['2024-10-29T23:00:00' 0.09772537607258255 0.14944471786532001 ... 0.0
  0.04093467143132262 0.10236877581152505]
 ['2024-10-30T00:00:00' 0.10110105647929389 0.1454553559488254 ... 0.0
  0.04751927963268399 0.1074755939909954]]


### Weather Data

In [2]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import importlib.util

# Function to check if required packages are installed
def check_package_installed(package_name):
    package_spec = importlib.util.find_spec(package_name)
    if package_spec is None:
        print(f"{package_name} is not installed!")
    else:
        print(f"{package_name} is installed!")

# Mapping of module names to package names
packages = {
    "openmeteo_requests": "openmeteo-requests",
    "requests_cache": "requests-cache",
    "retry_requests": "retry-requests",
}

for module_name, package_name in packages.items():
    check_package_installed(module_name)

# Setup the Open-Meteo API client with caching and retries
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Define the base URL for the weather API
url = "https://archive-api.open-meteo.com/v1/archive"

# Define the list of representative coordinates for Germany
coordinates = [
    {"latitude": 52.52, "longitude": 13.405},  # Berlin
    {"latitude": 53.5511, "longitude": 9.9937},  # Hamburg
    {"latitude": 48.1351, "longitude": 11.5820},  # Munich
    {"latitude": 50.9375, "longitude": 6.9603},  # Cologne
    {"latitude": 50.1109, "longitude": 8.6821},  # Frankfurt
    {"latitude": 51.0504, "longitude": 13.7373},  # Dresden
    {"latitude": 48.7758, "longitude": 9.1829},  # Stuttgart
]

# Define the weather variables and date range
params_template = {
    "start_date": "2018-01-01",
    "end_date": "2024-11-21",
    "hourly": [
        "temperature_2m", "relative_humidity_2m", "precipitation", 
        "surface_pressure", "cloud_cover", "wind_speed_100m", 
        "sunshine_duration", "shortwave_radiation", "direct_radiation"
    ]
}

# Store data for all locations
all_data = []

for coord in coordinates:
    params = params_template.copy()
    params.update({
        "latitude": coord["latitude"],
        "longitude": coord["longitude"],
    })

    try:
        # Fetch weather data for the current location
        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]

        # Extract hourly data for this location
        hourly = response.Hourly()
        hourly_data = {
            "date": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            )
        }
        hourly_data["temperature_2m"] = hourly.Variables(0).ValuesAsNumpy()
        hourly_data["relative_humidity_2m"] = hourly.Variables(1).ValuesAsNumpy()
        hourly_data["precipitation"] = hourly.Variables(2).ValuesAsNumpy()
        hourly_data["surface_pressure"] = hourly.Variables(3).ValuesAsNumpy()
        hourly_data["cloud_cover"] = hourly.Variables(4).ValuesAsNumpy()
        hourly_data["wind_speed_100m"] = hourly.Variables(5).ValuesAsNumpy()
        hourly_data["sunshine_duration"] = hourly.Variables(6).ValuesAsNumpy()
        hourly_data["shortwave_radiation"] = hourly.Variables(7).ValuesAsNumpy()
        hourly_data["direct_radiation"] = hourly.Variables(8).ValuesAsNumpy()

        # Convert to DataFrame and append to the list
        hourly_dataframe = pd.DataFrame(data=hourly_data)
        all_data.append(hourly_dataframe)

    except Exception as e:
        print(f"Error fetching data for coordinates {coord}: {e}")

# Combine all data into one DataFrame
combined_df = pd.concat(all_data)

# Group by date and calculate the mean for all variables
averaged_data = combined_df.groupby("date").mean()

# Rename columns for better understanding
averaged_data = averaged_data.rename(columns={
    "shortwave_radiation": "Global Horizontal Irradiance",
    "precipitation": "Precipitation (rain/snow)"
})

# Save the averaged data to a CSV file
csv_file = "../data/germany_weather_average.csv"
averaged_data.to_csv(csv_file, index=True)

print(f"Averaged weather data for Germany saved to {csv_file}.")


openmeteo_requests is installed!
requests_cache is installed!
retry_requests is installed!
Averaged weather data for Germany saved to ../data/germany_weather_average.csv.
